In [ ]:
import csv
import json

import os

In [ ]:
with open("geo/mable-raw.csv", "r", encoding="ansi") as infile:
    mableraw = list(csv.DictReader(infile))

In [ ]:
# MABLE data is *not* sorted properly, so we need to fix that.

mablegood = list(reversed(sorted(mableraw, key=lambda entry: (entry['afact'], entry['zcta']))))

In [ ]:
masterdict = {}
fipslookup = {}           # FIPS lookup table, to keep county names standardized in future processing
for row in mablegood:
    zcta = row['zcta']
    fipslookup[row['county']] = row['CountyName']
    if zcta not in ["", "ZIP census tabulation area", " "]:     # Drop bad row
        if zcta not in masterdict:     # Sorted list. Take the biggest entry first/highest population
            masterdict[zcta] = {
                # was auditee_zip, auditee_fips, auditee_county_name, auditee_zip_name
                "zip_code": zcta,
                "zip_fips": row['county'],
                "zip_county_name": row['CountyName'],
                "zip_place_name": row['ZIPName'],
            }
            

In [ ]:
with open("geo/hud-data.json", "r", encoding="utf-8") as infile:
    hudraw = json.load(infile)['data']['results']

In [ ]:
# HUD data is *not* sorted properly, so we need to fix that.

hudgood = list(reversed(sorted(hudraw, key=lambda entry: (entry['res_ratio'], entry['tot_ratio'], entry['zip']))))

In [ ]:
# HUD data is NOT sorted safely to know which county is predominate, as far as I can tell
# So let's try to sort through that

tally = {
    "total": 0,
    "duplicative": 0,
    "first_time": 0,
    "more_res": 0,
    "more_tot": 0,
    "inferior": 0,
}
hudlookup = {}
for entry in hudgood:
    tally['total'] += 1
    entryzip = entry['zip']
    if entryzip in masterdict:      # If we already have a Census Bureau entry, that wins and we do nothing else here
        tally['duplicative'] += 1
    else:
        if entryzip not in hudlookup:
            hudlookup[entryzip] = entry
            tally['first_time'] += 1
        else:             # We need this ZIP code but there's multiple counties
            remoteentry = hudlookup[entryzip]
            if entry['res_ratio'] > remoteentry['res_ratio']:        # More humans wins, every time
                hudlookup[entryzip] = entry
                tally['more_res'] += 1
                print(entryzip)
            elif (remoteentry['res_ratio'] == entry['res_ratio']) and entry['tot_ratio'] > remoteentry['tot_ratio']:
                hudlookup[entryzip] = entry
                tally['more_tot'] += 1
            else:
                tally['inferior'] += 1
print(f"{tally['total']:,} total entries found; {tally['duplicative']:,} duplicative; {tally['first_time']:,} first seen")
print(f"{tally['more_res']:,} had more residents than earlier entries; {tally['more_tot']:,} had more total addresses")
print(f"{tally['inferior']:,} were worse.")

In [ ]:
tally = {"found": 0, "missing": 0}
missingset = set()
for entryzip in hudlookup:
    entryfips = hudlookup[entryzip]['geoid']
    if entryfips in fipslookup:
        tally['found'] += 1
        hudlookup[entryzip]['zip_county_name'] = fipslookup[entryfips]
    else:
        tally['missing'] += 1
        missingset.add(entryfips)
        hudlookup[entryzip]['zip_county_name'] = "Unknown"
print(f"FIPS names found for {tally['found']:,} and missing for {tally['missing']:,} entries.")
print("FIPS codes missing for " + ' '.join(list(missingset)))

In [ ]:
# Census/geocorr/MABLE export does not include some entries for territories. Simon Willison's data does not include the
# abbreviation for the state. We could build another table, or we can do this stupidly.

fipspatches = {}
with open("geo/territories.json", "r", encoding="utf-8") as infile:
    territoriesraw = json.load(infile)
for entry in territoriesraw['rows']:
    fips = entry[1] + entry[2]
    placename = entry[6]
    if fips not in fipslookup:
        fipspatches[fips] = placename

In [ ]:
tally = {"patched": 0, "missing": 0}
for entryzip in hudlookup:
    if hudlookup[entryzip]['zip_county_name'] == "Unknown":
        if hudlookup[entryzip]['geoid'] not in fipspatches:
            print(f"Still missing entry for {hudlookup[entryzip]['geoid']}")
            tally['missing'] += 1
        else:
            hudlookup[entryzip]['zip_county_name'] = fipspatches[hudlookup[entryzip]['geoid']] + " " + hudlookup[entryzip]['state']
            tally['patched'] += 1
print(f"{tally['patched']:,} entries patched and {tally['missing']:,} entries remain missing.")

In [ ]:
hudlookup[entryzip]

In [ ]:
for entryzip in hudlookup:
    if entryzip in masterdict:
        print(f"How ... ? {entryzip}")
    else:
        entry = hudlookup[entryzip]
        line = {}
        line['zip_code'] = entryzip
        line['zip_fips'] = entry['geoid']
        line['zip_county_name'] = entry['zip_county_name']
        line['zip_place_name'] = entry['city']
        masterdict[entryzip] = line

In [ ]:
with open("geo/zip-lookup.csv", "w", encoding="utf-8", newline="") as outfile:
    writer = csv.writer(outfile)
    writer.writerow(list(masterdict['33467'].keys()))
    for zcta in sorted(masterdict):
        writer.writerow(list(masterdict[zcta].values()))